# Step 1 - Scrap Wikipedia for data

In [3]:

#install Beautiful Soup and requests for Web Scaping
!pip install BeautifulSoup4
!pip install requests

# Step 2: Get Data

In [12]:
#imports
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np


source = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(source.text, 'lxml')
data = []
columns = []
table = soup.find(class_='wikitable')
for index, tr in enumerate(table.find_all('tr')):
    section = []
    for td in tr.find_all(['th','td']):
        section.append(td.text.rstrip())

    if (index == 0):
        columns = section
    else:
        data.append(section)

canadapostcode_df = pd.DataFrame(data = data,columns = columns)
canadapostcode_df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


# Step 3 : Clean Data

In [13]:
canadapostcode_df = canadapostcode_df[canadapostcode_df['Borough'] != 'Not assigned']
canadapostcode_df.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


In [14]:
canadapostcode_df["Neighbourhood"] = canadapostcode_df.groupby("Postcode")["Neighbourhood"].transform(lambda neigh: ', '.join(neigh))
canadapostcode_df = canadapostcode_df.drop_duplicates()
if(canadapostcode_df.index.name != 'Postcode'):
   canadapostcode_df = canadapostcode_df.set_index('Postcode') 
canadapostcode_df.head()

,Borough,Neighbourhood
Postcode,,
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,"Harbourfront, Regent Park"
M6A,North York,"Lawrence Heights, Lawrence Manor"
M7A,Queen's Park,Not assigned


In [15]:
canadapostcode_df['Neighbourhood'].replace("Not assigned", canadapostcode_df["Borough"],inplace=True)
canadapostcode_df.head()

,Borough,Neighbourhood
Postcode,,
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,"Harbourfront, Regent Park"
M6A,North York,"Lawrence Heights, Lawrence Manor"
M7A,Queen's Park,Queen's Park


# Step 3:  Number of Rows of the Data Frame


In [16]:
canadapostcode_df.shape

(103, 2)

In [18]:
geospat_coord_df = pd.read_csv('http://cocl.us/Geospatial_data')

#rename columns and set the index to be Postcode
geospat_coord_df.columns = ["Postcode", "Latitude", "Longitude"]
if(geospat_coord_df.index.name != 'Postcode'):
    geospat_coord_df = geospat_coord_df.set_index('Postcode')
    
geospat_coord_df.head()

,Latitude,Longitude
Postcode,,
M1B,43.806686,-79.194353
M1C,43.784535,-79.160497
M1E,43.763573,-79.188711
M1G,43.770992,-79.216917
M1H,43.773136,-79.239476


# Step 4: Use of GeoSpatial Data

In [24]:
canadapostcode_with_geospat_df = canadapostcode_with_geospat_df.reset_index()
canadapostcode_with_geospat_df.head(12)

,index,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,0,M3A,North York,Parkwoods,43.753259,-79.329656
1,1,M4A,North York,Victoria Village,43.725882,-79.315572
2,2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
3,3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
5,5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,6,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
7,7,M3B,North York,Don Mills North,43.745906,-79.352188
8,8,M4B,East York,"Woodbine Gardens, Parkview Hill",43.706397,-79.309937
9,9,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937


In [20]:
canadapostcode_with_geospat_df = canadapostcode_df.join(geospat_coord_df)
canadapostcode_with_geospat_df.head(11)

,Borough,Neighbourhood,Latitude,Longitude
Postcode,,,,
M3A,North York,Parkwoods,43.753259,-79.329656
M4A,North York,Victoria Village,43.725882,-79.315572
M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
M7A,Queen's Park,Queen's Park,43.662301,-79.389494
M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
M3B,North York,Don Mills North,43.745906,-79.352188
M4B,East York,"Woodbine Gardens, Parkview Hill",43.706397,-79.309937
